### Deep Learning Up and Running
#### By MiMoTrix
##### MNIST with FullyConnected Network

In [19]:
import tensorflow as tf
import time


In [12]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
learning_rate = 0.01
epochs = 30
batch_size = 100

##### Weight Variables Initialize

In [18]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

##### Bias Variables Initialize

In [20]:
def bias_variable(shape):
    initial = tf.constant(0.0, shape=shape)
    return tf.Variable(initial)

##### Inputs

In [37]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

KeepProbInput = tf.placeholder(tf.float32) # Against Biasing
X_Drop = tf.nn.dropout(X, keep_prob=KeepProbInput)
Weights_HL_One = weight_variable([784, 500])
Biases_HL_One = bias_variable([500])
Output_HL_One = tf.nn.relu(tf.matmul(X_Drop, Weights_HL_One) + Biases_HL_One)

KeepProb = tf.placeholder(tf.float32) # Against Biasing
Output_Drop_LayerOne = tf.nn.dropout(Output_HL_One, keep_prob=KeepProb)
Weights_HL_Two = weight_variable([500, 500])
Biases_HL_Two = bias_variable([500])
Output_HL_Two = tf.nn.relu(tf.matmul(Output_Drop_LayerOne, Weights_HL_Two) + Biases_HL_Two)

Output_Drop_LayerTwo = tf.nn.dropout(Output_HL_Two, KeepProbInput)
Weights_HL_Output = weight_variable([500, 10])
Biases_HL_Output = bias_variable([10])
Y_Predicted = tf.nn.softmax(tf.matmul(Output_Drop_LayerTwo, Weights_HL_Output) + Biases_HL_Output)

##### Cross Entropy, Loss Function, Accuaracy, etc.

In [38]:
crossEntropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_Predicted))
trainStep = tf.train.GradientDescentOptimizer(learning_rate).minimize(crossEntropy)
correctPrediction = tf.equal(tf.argmax(Y_Predicted, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correctPrediction, tf.float32))


##### Session Run 

In [39]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batchSize = 100
print("Startin Burn-In...")

Startin Burn-In...


##### Execute

In [ ]:
for i in range(700):
    input_images, correct_predictions = mnist.train.next_batch(batchSize)
    if i % (60000/batch_size) == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
            X: input_images, Y: correct_predictions, KeepProbInput: 1.0, KeepProb: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
        # validate
        test_accuracy = sess.run(accuracy, feed_dict={
            X: mnist.test.images, Y: mnist.test.labels, KeepProbInput: 1.0, KeepProb: 1.0})
        print("Validation accuracy: %g." % test_accuracy)
    sess.run(trainStep, feed_dict={X: input_images, Y: correct_predictions, KeepProbInput: 0.8, KeepProb: 0.5})
print("Starting the training...")
start_time = time.time()
best_accuracy = 0.0
for i in range(int((20*60000)/batchSize)):
    input_images, correct_predictions = mnist.train.next_batch(batch_size)
    if i % (60000/batch_size) == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
            X: input_images, Y: correct_predictions, KeepProbInput: 1.0, KeepProb: 1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
        # validate
        test_accuracy = sess.run(accuracy, feed_dict={
            X: mnist.test.images, Y: mnist.test.labels, KeepProbInput: 1.0, KeepProb: 1.0})
        if test_accuracy >= best_accuracy:
            best_accuracy = test_accuracy
            print("Validation accuracy improved: %g." % test_accuracy)
        else:
            print("Validation accuracy was: %g. It was better before: %g. " % (test_accuracy, best_accuracy) +
                  "Using the old params for further optimizations.")
    sess.run(trainStep, feed_dict={X: input_images, Y: correct_predictions, KeepProbInput: 0.8, KeepProb: 0.5})
print("The training took %.4f seconds." % (time.time() - start_time))

# validate
print("Best test accuracy: %g" % best_accuracy)

step 0, training accuracy 0.17


Validation accuracy: 0.1135.


step 600, training accuracy 0.06


Validation accuracy: 0.1135.


Starting the training...
step 0, training accuracy 0.14


...

step 10800, training accuracy 0.2


Validation accuracy was: 0.2011. It was better before: 0.2254. Using the old params for further optimizations.


step 11400, training accuracy 0.19


Validation accuracy was: 0.1886. It was better before: 0.2254. Using the old params for further optimizations.


The training took 67.0380 seconds.
Best test accuracy: 0.2254
